In [ ]:
#imports
import pandas as pd
from aitextgen import aitextgen
import ipywidgets as widgets

In [ ]:
# pandas options for bigger grids
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [ ]:
# Download and load the model. 
# Set to_gpu=False if running on a CPU.
ai = aitextgen(model="minimaxir/magic-the-gathering", to_gpu=False)


In [ ]:
# widgets
from ipywidgets import HBox, VBox, Label, Layout

wider = Layout(width="70vw")
less_wide = Layout(width="5vw")
promptInput = widgets.Text(placeholder='ex: <|type|>Creature - Goblin<|manaCost|>{2}{R}', layout=wider)
crazinessSlider = widgets.FloatSlider(.7,min=0.5,max=1.3)
numOfCardsInput = widgets.BoundedIntText(3,min=1,max=100,layout=less_wide)

display(HBox([Label("custom card prompt:"), promptInput]))
display(VBox([
    Label("(prompt list: <|name|>, <|manaCost|>, <|type|>, <|text|>,"),
    Label("<|power|>, <|toughness|>, <|loyalty|>)")
]))
display(HBox([Label("craziness:"), crazinessSlider]))
display(HBox([Label("number of cards to generate:"), numOfCardsInput]))

comment or remove the first line in the next block to continue

In [ ]:
# return;
prompt=promptInput.value
craziness = crazinessSlider.value
numOfCards = batch_size = numOfCardsInput.value

cards = ai.generate(n=numOfCards,
                    batch_size=batch_size,
                    schema=True,
                    prompt=prompt,
                    temperature=craziness,
                    return_as_list=True)

cards = pd.DataFrame(list(cards))
cards['text'] = cards.apply(lambda row: row['text'].replace("~", row['name']), axis=1)
cards = cards[['name', 'manaCost', 'type', 'power', 'toughness','loyalty','text']]
display(cards)

In [ ]:
saveCardsBtn = widgets.Button(description="save cards")
fileLocationInput = widgets.Text(value="My Cards.csv")

def saveCards(b):
    cards.to_csv(f'./output/{fileLocationInput.value}')
    pass

saveCardsBtn.on_click(saveCards)
display(HBox([fileLocationInput, saveCardsBtn]))